In [56]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

In [2]:
drug_interaction=pd.read_csv("drug_interaction_drugcomb.csv",low_memory=False)
drug_feature=pd.read_csv("drug_feature.csv",index_col=0)
cell_feature=pd.read_csv("final_cell_line_feature.csv",index_col=0)

In [3]:
drug_interaction.shape

(1048575, 18)

In [4]:
drug_interaction.head()

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
0,1,5-FU,ABT-888,A2058,uM,uM,30.869,3.865915,6.256584,-2.951386,5.536903,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.839
1,2,5-FU,ABT-888,A2058,uM,uM,27.460,8.247403,12.333896,3.125927,11.614215,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.430
2,3,5-FU,ABT-888,A2058,uM,uM,29.901,6.063440,11.660209,2.452239,10.940528,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.871
3,4,5-FU,ABT-888,A2058,uM,uM,24.016,-4.280231,5.145209,-4.062761,4.425528,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.986
4,5,5-FU,AZD1775,A2058,uM,uM,66.847,12.284698,15.765467,10.409407,18.656340,5.126836,0.266027,11.471,25.164,76.501,57.193,30.212


In [5]:
drug_interaction.dropna(inplace=True)

In [6]:
drug_interaction.shape

(466033, 18)

In [7]:
drug_interaction['synergistic_status'] = drug_interaction['synergy_zip'].apply(lambda x: 1 if x > 0 else 0)

In [8]:
drug_interaction=drug_interaction[['drug_row','drug_col','cell_line_name','synergistic_status']]

In [9]:
drug_interaction.head()

,drug_row,drug_col,cell_line_name,synergistic_status
0,5-FU,ABT-888,A2058,1
1,5-FU,ABT-888,A2058,1
2,5-FU,ABT-888,A2058,1
3,5-FU,ABT-888,A2058,0
4,5-FU,AZD1775,A2058,1


In [10]:
drug_interaction.synergistic_status.value_counts()

synergistic_status
0    258510
1    207523
Name: count, dtype: int64

In [11]:
interaction_drug= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [12]:
len(interaction_drug)

4150

In [13]:
drug_feature.head()

,drug,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5-FU,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABT-888,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AZD1775,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BEZ-235,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BORTEZOMIB,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
all_drug_with_feature=set(drug_feature['drug'].unique())

In [15]:
len(all_drug_with_feature)

4051

##### if all interaction drug has feature or not

In [16]:
if interaction_drug.issubset(all_drug_with_feature):
    print("All interaction drugs are present in the feature set.")
else:
    print(" Some interaction drugs are missing in the feature set.")
    missing = interaction_drug - all_drug_with_feature
    print("Missing drugs:", missing)


 Some interaction drugs are missing in the feature set.
Missing drugs: {'ZINC17545571', "3',4'-Diacetylafzelin", 'ACMC-20mj2l', 'CTK8F0346', 'MOLPORT-042-665-727', 'AC1OFCJY', 'TETRAETHYLENEPENTAMINE 5HCL', 'KS-00001D5I', 'DSSTox_CID_28582', 'AC1L1EUK', 'ZINC44963243', 'AC1LIMNP', 'AK174336', 'AK160201', 'QCR-20', 'AK-77283', 'ZINC208949883', 'PIFITHRIN-A (PFTA)', 'ZINC3881972', 'Vincristine Sulfate, Apocynaceae sp.', 'ZINC139045355', 'ZINC208951860', 'AC1OFCEV', 'Triciribinephosphate', 'ZINC250325905', 'AC1OF340', 'AC1NUNML', 'ZINC896472', 'AC1MWLHC', 'ZINC11616261', 'ZINC97975747', 'AC1O7H3F', 'Curcumin, Curcuma longa L.', 'ZINC43163828', 'ERK 11e', 'AK198640', 'ONO-4059 analog', 'ZINC101112865', 'ZINC18182079', 'ZINC11616871', 'AC1LCZJT', 'AC1L9B3J', 'AC1L1KJK', 'KS-00001CUZ', 'Pd-1/pd-l1 inhibitor 1', 'KGBPLKOPSFDBOX-UHFFFAOYSA-N', 'DSSTox_CID_31408', 'ZINC21999985', 'AC1LAYFC', "Phloretin 2'-galactoside", 'ZINC84688828', '1beta-Methylimipenem', 'ZINC3831133', 'AC1L21NN', 'AC1M4BZH

In [17]:
drug_interaction = drug_interaction[
    ~drug_interaction['drug_row'].isin(missing) &
    ~drug_interaction['drug_col'].isin(missing)
].reset_index(drop=True)

In [18]:
drug_interaction.shape

(464744, 4)

In [19]:
all_drugs_before = set(drug_interaction['drug_row']) | set(drug_interaction['drug_col'])
all_drugs_after = set(drug_interaction['drug_row']) | set(drug_interaction['drug_col'])

removed_drugs = all_drugs_before - all_drugs_after
print(f"Number of unique drugs before: {len(all_drugs_before)}")
print(f"Number of unique drugs after: {len(all_drugs_after)}")
print(f"Drugs removed: {len(removed_drugs)}")
print(f"Drugs removed list: {removed_drugs}")

Number of unique drugs before: 4051
Number of unique drugs after: 4051
Drugs removed: 0
Drugs removed list: set()


In [20]:
interaction_drug= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [21]:
interaction_drug_new= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [22]:
len(interaction_drug),len(interaction_drug_new)

(4051, 4051)

In [23]:
all_drug_with_feature==interaction_drug_new

True

In [24]:
drug_feature['drug_id'] = range(len(drug_feature))

In [25]:
drug_to_id = dict(zip(drug_feature['drug'], drug_feature['drug_id']))

In [26]:
drug_feature.drop(columns=['drug_id','drug'],inplace=True)

In [27]:
drug_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
drug_feature.shape

(4051, 2048)

In [29]:
drug_interaction['drug_row_id'] = drug_interaction['drug_row'].map(drug_to_id)
drug_interaction['drug_col_id'] = drug_interaction['drug_col'].map(drug_to_id)

In [30]:
drug_interaction = drug_interaction.drop(columns=['drug_row', 'drug_col'])

In [31]:
cols = ['drug_row_id', 'drug_col_id'] + [col for col in drug_interaction.columns if col not in ['drug_row_id', 'drug_col_id']]
drug_interaction = drug_interaction[cols]

In [32]:
drug_interaction.head()

,drug_row_id,drug_col_id,cell_line_name,synergistic_status
0,0,1,A2058,1
1,0,1,A2058,1
2,0,1,A2058,1
3,0,1,A2058,0
4,0,2,A2058,1


In [33]:
drug_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
cos_sim_matrix = cosine_similarity(drug_feature)

In [35]:
cos_sim_matrix

array([[1.        , 0.17699808, 0.11211037, ..., 0.11646819, 0.09727208,
        0.14808722],
       [0.17699808, 1.        , 0.2073627 , ..., 0.28200837, 0.22898571,
        0.21912525],
       [0.11211037, 0.2073627 , 1.        , ..., 0.17366199, 0.20719896,
        0.34698416],
       ...,
       [0.11646819, 0.28200837, 0.17366199, ..., 1.        , 0.21525296,
        0.27526932],
       [0.09727208, 0.22898571, 0.20719896, ..., 0.21525296, 1.        ,
        0.26000576],
       [0.14808722, 0.21912525, 0.34698416, ..., 0.27526932, 0.26000576,
        1.        ]], shape=(4051, 4051))

In [36]:
cos_sim_matrix.shape

(4051, 4051)

In [37]:
interaction_drug= set(drug_interaction['drug_row_id']).union(set(drug_interaction['drug_col_id']))

In [38]:
len(interaction_drug)

4051

In [39]:
drug_feature.shape

(4051, 2048)

In [40]:
df_synergy = drug_interaction[drug_interaction['synergistic_status'] == 1]


In [41]:
df_synergy

,drug_row_id,drug_col_id,cell_line_name,synergistic_status
0,0,1,A2058,1
1,0,1,A2058,1
2,0,1,A2058,1
4,0,2,A2058,1
5,0,2,A2058,1
...,...,...,...,...
464739,4046,1823,KB-ChR-8-5-11,1
464740,4047,1823,KB-ChR-8-5-11,1
464741,4048,1823,KB-ChR-8-5-11,1
464742,4049,1823,KB-ChR-8-5-11,1


In [42]:
num_drugs = max(drug_interaction['drug_row_id'].max(), drug_interaction['drug_col_id'].max()) + 1


In [43]:
num_drugs

np.int64(4051)

In [44]:
adj_matrix = np.zeros((num_drugs, num_drugs))


In [45]:
adj_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4051, 4051))

In [46]:
for _, row in df_synergy.iterrows():
    i = int(row['drug_row_id'])
    j = int(row['drug_col_id'])
    adj_matrix[i, j] = 1
    adj_matrix[j, i] = 1  

In [47]:
np.fill_diagonal(adj_matrix, 0)

In [48]:
adj_matrix.shape

(4051, 4051)

In [49]:
adj_matrix

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4051, 4051))

In [50]:
cos_sim_matrix

array([[1.        , 0.17699808, 0.11211037, ..., 0.11646819, 0.09727208,
        0.14808722],
       [0.17699808, 1.        , 0.2073627 , ..., 0.28200837, 0.22898571,
        0.21912525],
       [0.11211037, 0.2073627 , 1.        , ..., 0.17366199, 0.20719896,
        0.34698416],
       ...,
       [0.11646819, 0.28200837, 0.17366199, ..., 1.        , 0.21525296,
        0.27526932],
       [0.09727208, 0.22898571, 0.20719896, ..., 0.21525296, 1.        ,
        0.26000576],
       [0.14808722, 0.21912525, 0.34698416, ..., 0.27526932, 0.26000576,
        1.        ]], shape=(4051, 4051))

In [51]:
drug_feature=np.array(drug_feature)

In [57]:
X = torch.tensor(drug_feature, dtype=torch.float32)        
A_label = torch.tensor(adj_matrix, dtype=torch.float32)
A_cos = torch.tensor(cos_sim_matrix, dtype=torch.float32)

In [58]:
A_label.shape

torch.Size([4051, 4051])

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X = X.to(device)
A_label = A_label.to(device)



In [60]:
def normalize_adj(adj):
    adj = adj + torch.eye(adj.shape[0]).to(device)  # Add self-loops
    deg = torch.sum(adj, dim=1)
    deg_inv_sqrt = torch.pow(deg, -0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0.0
    D_inv_sqrt = torch.diag(deg_inv_sqrt)
    return D_inv_sqrt @ adj @ D_inv_sqrt

norm_adj = normalize_adj(A_label)

# INITIAL MODEL

In [62]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GAE, self).__init__()
        self.W = nn.Parameter(torch.randn(in_dim, hidden_dim) * 0.01)

    def encode(self, X, norm_adj):
        return torch.relu(norm_adj @ X @ self.W)  # GCN layer

    def decode(self, Z):
        return torch.sigmoid(Z @ Z.T)  # Inner-product decoder

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [63]:
model = GAE(in_dim=2048, hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [65]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [69]:
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    
    loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 002 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 003 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 004 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 005 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 006 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 007 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 008 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 009 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 010 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 011 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 012 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 013 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 014 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 015 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 016 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 017 | Loss: 0.6931 | ROC-AUC: 0.50

In [67]:
density = A_label.sum() / (A_label.shape[0] ** 2)
print(f"Sparsity: {density.item():.6f}")


Sparsity: 0.003546


# WITH SELF LOOP

In [70]:
A_label = A_label + torch.eye(A_label.shape[0]).to(device)


In [73]:
def normalize_adj(adj):
    adj = adj + torch.eye(adj.shape[0]).to(device)  # Add self-loops
    deg = torch.sum(adj, dim=1)
    deg_inv_sqrt = torch.pow(deg, -0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0.0
    D_inv_sqrt = torch.diag(deg_inv_sqrt)
    return D_inv_sqrt @ adj @ D_inv_sqrt

norm_adj = normalize_adj(A_label)

In [74]:

true_edges = A_label[triu_indices[0], triu_indices[1]]

In [75]:
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    
    loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 002 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 003 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 004 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 005 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 006 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 007 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 008 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 009 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 010 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 011 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 012 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 013 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 014 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 015 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 016 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 017 | Loss: 0.6931 | ROC-AUC: 0.50

# ReLU TO LeakyReLU IN ENCODER

In [76]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GAE, self).__init__()
        self.W = nn.Parameter(torch.randn(in_dim, hidden_dim) * 0.01)

    def encode(self, X, norm_adj):
        return F.leaky_relu(norm_adj @ X @ self.W, negative_slope=0.01)


    def decode(self, Z):
        return torch.sigmoid(Z @ Z.T)  # Inner-product decoder

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [77]:
model = GAE(in_dim=2048, hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [78]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [79]:
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    
    loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 0.7170 | ROC-AUC: 0.8058 | PR-AUC: 0.0935
Epoch 002 | Loss: 0.6938 | ROC-AUC: 0.9300 | PR-AUC: 0.1327
Epoch 003 | Loss: 0.6949 | ROC-AUC: 0.9298 | PR-AUC: 0.1326
Epoch 004 | Loss: 0.6960 | ROC-AUC: 0.9297 | PR-AUC: 0.1326
Epoch 005 | Loss: 0.6972 | ROC-AUC: 0.9297 | PR-AUC: 0.1326
Epoch 006 | Loss: 0.6983 | ROC-AUC: 0.9296 | PR-AUC: 0.1326
Epoch 007 | Loss: 0.6993 | ROC-AUC: 0.9296 | PR-AUC: 0.1325
Epoch 008 | Loss: 0.7002 | ROC-AUC: 0.9295 | PR-AUC: 0.1325
Epoch 009 | Loss: 0.7009 | ROC-AUC: 0.9295 | PR-AUC: 0.1325
Epoch 010 | Loss: 0.7018 | ROC-AUC: 0.9273 | PR-AUC: 0.1273
Epoch 011 | Loss: 0.7022 | ROC-AUC: 0.9295 | PR-AUC: 0.1325
Epoch 012 | Loss: 0.7027 | ROC-AUC: 0.9290 | PR-AUC: 0.1317
Epoch 013 | Loss: 0.7032 | ROC-AUC: 0.9270 | PR-AUC: 0.1299
Epoch 014 | Loss: 0.7034 | ROC-AUC: 0.9293 | PR-AUC: 0.1320
Epoch 015 | Loss: 0.7036 | ROC-AUC: 0.9293 | PR-AUC: 0.1322
Epoch 016 | Loss: 0.7038 | ROC-AUC: 0.9293 | PR-AUC: 0.1324
Epoch 017 | Loss: 0.7039 | ROC-AUC: 0.92

# USE BCEWithLogitsLoss

In [82]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GAE, self).__init__()
        self.W = nn.Parameter(torch.randn(in_dim, hidden_dim) * 0.01)

    def encode(self, X, norm_adj):
        return F.leaky_relu(norm_adj @ X @ self.W, negative_slope=0.01)


    def decode(self, Z):
        return  Z @ Z.T  # Inner-product decoder
  
        

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [83]:
model = GAE(in_dim=2048, hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [84]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [87]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    
    # loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 1.1430 | ROC-AUC: 0.9329 | PR-AUC: 0.1354
Epoch 002 | Loss: 1.1386 | ROC-AUC: 0.9333 | PR-AUC: 0.1357
Epoch 003 | Loss: 1.1351 | ROC-AUC: 0.9337 | PR-AUC: 0.1360
Epoch 004 | Loss: 1.1324 | ROC-AUC: 0.9341 | PR-AUC: 0.1363
Epoch 005 | Loss: 1.1304 | ROC-AUC: 0.9346 | PR-AUC: 0.1367
Epoch 006 | Loss: 1.1290 | ROC-AUC: 0.9352 | PR-AUC: 0.1371
Epoch 007 | Loss: 1.1280 | ROC-AUC: 0.9358 | PR-AUC: 0.1375
Epoch 008 | Loss: 1.1273 | ROC-AUC: 0.9364 | PR-AUC: 0.1380
Epoch 009 | Loss: 1.1268 | ROC-AUC: 0.9371 | PR-AUC: 0.1385
Epoch 010 | Loss: 1.1265 | ROC-AUC: 0.9378 | PR-AUC: 0.1391
Epoch 011 | Loss: 1.1262 | ROC-AUC: 0.9385 | PR-AUC: 0.1396
Epoch 012 | Loss: 1.1259 | ROC-AUC: 0.9393 | PR-AUC: 0.1403
Epoch 013 | Loss: 1.1255 | ROC-AUC: 0.9401 | PR-AUC: 0.1409
Epoch 014 | Loss: 1.1251 | ROC-AUC: 0.9409 | PR-AUC: 0.1416
Epoch 015 | Loss: 1.1245 | ROC-AUC: 0.9418 | PR-AUC: 0.1423
Epoch 016 | Loss: 1.1237 | ROC-AUC: 0.9426 | PR-AUC: 0.1431
Epoch 017 | Loss: 1.1228 | ROC-AUC: 0.94

In [ ]:
Learnig=ng rate adj

In [ ]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GAE, self).__init__()
        self.W = nn.Parameter(torch.empty(in_dim, hidden_dim))
        nn.init.xavier_uniform_(self.W)

    def encode(self, X, norm_adj):
        return F.leaky_relu(norm_adj @ X @ self.W, negative_slope=0.01)


    def decode(self, Z):
        return Z @ Z.T  # Inner-product decoder
  
        

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [ ]:
model = GAE(in_dim=2048, hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [88]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [89]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    
    # loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 0.9628 | ROC-AUC: 0.9851 | PR-AUC: 0.1859
Epoch 002 | Loss: 0.9624 | ROC-AUC: 0.9851 | PR-AUC: 0.1859
Epoch 003 | Loss: 0.9620 | ROC-AUC: 0.9851 | PR-AUC: 0.1860
Epoch 004 | Loss: 0.9616 | ROC-AUC: 0.9852 | PR-AUC: 0.1861
Epoch 005 | Loss: 0.9612 | ROC-AUC: 0.9852 | PR-AUC: 0.1861
Epoch 006 | Loss: 0.9608 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 007 | Loss: 0.9604 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 008 | Loss: 0.9600 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 009 | Loss: 0.9596 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 010 | Loss: 0.9592 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 011 | Loss: 0.9588 | ROC-AUC: 0.9854 | PR-AUC: 0.1860
Epoch 012 | Loss: 0.9584 | ROC-AUC: 0.9854 | PR-AUC: 0.1860
Epoch 013 | Loss: 0.9581 | ROC-AUC: 0.9854 | PR-AUC: 0.1860
Epoch 014 | Loss: 0.9577 | ROC-AUC: 0.9854 | PR-AUC: 0.1861
Epoch 015 | Loss: 0.9574 | ROC-AUC: 0.9854 | PR-AUC: 0.1861
Epoch 016 | Loss: 0.9570 | ROC-AUC: 0.9854 | PR-AUC: 0.1860
Epoch 017 | Loss: 0.9567 | ROC-AUC: 0.98

# implementing deep GAE

In [90]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim1, hidden_dim2, dropout=0.3):
        super(GAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, hidden_dim1))
        self.W2 = nn.Parameter(torch.empty(hidden_dim1, hidden_dim2))
        
        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        return h2  # final node embedding Z

    def decode(self, Z):
        return Z @ Z.T  # Inner-product decoder

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z


In [92]:
model = GAE(in_dim=2048, hidden_dim1=512, hidden_dim2=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [93]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [94]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    
    # loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 1.2613 | ROC-AUC: 0.7227 | PR-AUC: 0.0969
Epoch 002 | Loss: 1.2701 | ROC-AUC: 0.9063 | PR-AUC: 0.1608
Epoch 003 | Loss: 1.1121 | ROC-AUC: 0.9693 | PR-AUC: 0.1769
Epoch 004 | Loss: 1.8424 | ROC-AUC: 0.9725 | PR-AUC: 0.2163
Epoch 005 | Loss: 1.2862 | ROC-AUC: 0.8210 | PR-AUC: 0.0512
Epoch 006 | Loss: 1.2912 | ROC-AUC: 0.9024 | PR-AUC: 0.1483
Epoch 007 | Loss: 1.2052 | ROC-AUC: 0.9571 | PR-AUC: 0.1200
Epoch 008 | Loss: 1.1555 | ROC-AUC: 0.9694 | PR-AUC: 0.1365
Epoch 009 | Loss: 1.2478 | ROC-AUC: 0.9628 | PR-AUC: 0.1970
Epoch 010 | Loss: 1.0783 | ROC-AUC: 0.9737 | PR-AUC: 0.1808
Epoch 011 | Loss: 1.1382 | ROC-AUC: 0.9794 | PR-AUC: 0.2112
Epoch 012 | Loss: 1.0955 | ROC-AUC: 0.9816 | PR-AUC: 0.2112
Epoch 013 | Loss: 1.0426 | ROC-AUC: 0.9813 | PR-AUC: 0.2123
Epoch 014 | Loss: 1.0240 | ROC-AUC: 0.9864 | PR-AUC: 0.2066
Epoch 015 | Loss: 1.0112 | ROC-AUC: 0.9864 | PR-AUC: 0.2285
Epoch 016 | Loss: 1.0216 | ROC-AUC: 0.9874 | PR-AUC: 0.2553
Epoch 017 | Loss: 1.0092 | ROC-AUC: 0.99

In [ ]:
# After training
model.eval()
with torch.no_grad():
    A_pred_final, Z_final = model(X, norm_adj)  # [4051×4051], [4051×128]
    A_pred_np = A_pred_final.cpu().numpy()      # Final decoded matrix
    Z_np = Z_final.cpu().numpy() 